In [25]:
import numpy as np
import fuzzy
import matplotlib.pyplot as plt
import random

In [2]:
fis = fuzzy.mandani()
fis.add_var({"type":"input","name":"pop","range":[0.0,1.0]})
fis.add_var({"type":"input","name":"fav","range":[0.0,1.0]})
fis.add_var({"type":"output","name":"ctrl","range":[0.0,0.5]})

fis.add_mfs({"vartype":"input","varname":"pop","mfname":"BI","params":[0.0,0.0,0.20]})
fis.add_mfs({"vartype":"input","varname":"pop","mfname":"B","params":[0.0,0.20,0.4]})
fis.add_mfs({"vartype":"input","varname":"pop","mfname":"M","params":[0.2,0.4,0.6]})
fis.add_mfs({"vartype":"input","varname":"pop","mfname":"MA","params":[0.4,0.6,0.8]})
fis.add_mfs({"vartype":"input","varname":"pop","mfname":"A","params":[0.6,0.8,1.0]})
fis.add_mfs({"vartype":"input","varname":"pop","mfname":"AT","params":[0.8,1.0,1.0]})


fis.add_mfs({"vartype":"input","varname":"fav","mfname":"BI","params":[0.0,0.0,0.20]})
fis.add_mfs({"vartype":"input","varname":"fav","mfname":"B","params":[0.0,0.20,0.4]})
fis.add_mfs({"vartype":"input","varname":"fav","mfname":"M","params":[0.2,0.4,0.6]})
fis.add_mfs({"vartype":"input","varname":"fav","mfname":"MA","params":[0.4,0.6,0.8]})
fis.add_mfs({"vartype":"input","varname":"fav","mfname":"A","params":[0.6,0.8,1.0]})
fis.add_mfs({"vartype":"input","varname":"fav","mfname":"AT","params":[0.8,1.0,1.0]})


fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"BI","params":[0.0,0.0,0.10]})
fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"B","params":[0.0,0.10,0.2]})
fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"M","params":[0.1,0.2,0.3]})
fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"MA","params":[0.2,0.3,0.4]})
fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"A","params":[0.3,0.4,0.5]})
fis.add_mfs({"vartype":"output","varname":"ctrl","mfname":"AT","params":[0.4,0.5,0.5]})



#cria regras
for p in ["BI","B","M","MA","A","AT"]:
    for f in ["BI","B","M","MA","A","AT"]:
        fis.add_rule({"pop":p,"fav":f},{"ctrl":"BI"})


In [37]:
def dinamica(to,po,n):
    p = [po]
    fcn = lambda t: 1-0.5*np.cos(2*np.pi*t/12)
    for i in range(n):
        fav = 0.10*fcn(to+i)
        res = fis.eval([p[i],fav]) 
        pn = (p[i]+fav)*(1-res["ctrl"])
        p.append(pn)
    
    return p


def dinamica2(to,po,n,fis):
    p = [po]
    fcn = lambda t: 1-0.5*np.cos(2*np.pi*t/12)
    u = [0]
    for i in range(n):
        fav = 0.10*fcn(to+i)
        res = fis.eval([p[i],fav]) 
        pn = (p[i]+fav)*(1-res["ctrl"])
        p.append(pn)
        u.append(res["ctrl"])
    
    return p,u

labels = ["BI","B","M","MA","A","AT"]
def objetivo(x):
    fis.rules = {}
    cnt = 0
    for p in labels:
        for f in labels:
            fis.add_rule({"pop":p,"fav":f},{"ctrl":labels[x[cnt]]})
            cnt = cnt+1
    
    pp,uu = dinamica2(0,0.05,12,fis)

    obj = (pp[-1]-0.3)**2+0.1*sum([u**2 for u in uu ])

    return obj
    


In [44]:
obj = objetivo([5]*36)
obj

0.30170478697276837

In [4]:
from pyomo.environ import *

In [38]:
# Cria um modelo
modelo = ConcreteModel()
modelo.x = Var(range(36), domain=Integers, bounds=(0, 5))
valores_iniciais = [random.randint(0,5) for i in range(36)]   # Por exemplo, atribuir o valor 1 a todas as variáveis inicialmente
for i in range(36):
    modelo.x[i].value = valores_iniciais[i]
modelo.obj= Objective(rule=lambda modelo: objetivo([value(modelo.x[i]) for i in range(36)]), sense=minimize)
solver = SolverFactory('glpk')
resultado = solver.solve(modelo)


In [39]:
print("Status:", resultado.solver.termination_condition)

# Imprima os valores das variáveis
for i in range(36):
    print("x[{}] = {}".format(i, value(modelo.x[i])))

# Imprima o valor da função objetivo
print("Valor da função objetivo:", value(modelo.obj))

Status: optimal
x[0] = 3
x[1] = 5
x[2] = 5
x[3] = 3
x[4] = 5
x[5] = 3
x[6] = 0
x[7] = 2
x[8] = 5
x[9] = 3
x[10] = 1
x[11] = 5
x[12] = 3
x[13] = 2
x[14] = 4
x[15] = 4
x[16] = 5
x[17] = 5
x[18] = 5
x[19] = 2
x[20] = 5
x[21] = 3
x[22] = 2
x[23] = 1
x[24] = 3
x[25] = 4
x[26] = 5
x[27] = 5
x[28] = 4
x[29] = 5
x[30] = 3
x[31] = 3
x[32] = 4
x[33] = 0
x[34] = 0
x[35] = 5
Valor da função objetivo: 0.06439608136008257


In [40]:
valores_iniciais

[3,
 5,
 5,
 3,
 5,
 3,
 0,
 2,
 5,
 3,
 1,
 5,
 3,
 2,
 4,
 4,
 5,
 5,
 5,
 2,
 5,
 3,
 2,
 1,
 3,
 4,
 5,
 5,
 4,
 5,
 3,
 3,
 4,
 0,
 0,
 5]